## Movie_Review_Sentiment_Classification
### (영화리뷰 감성분석)

***
#### 1. 데이터 준비와 확인

In [2]:
import pandas as pd
import tensorflow as tf
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


***  
#### 2. 데이터로더 구성
> **1) 데이터의 중복 제거  
2) NaN 결측치 제거  
3) 한국어 토크나이저로 토큰화  
4) 불용어(Stopwords) 제거  
5) 사전word_to_index 구성  
6) 텍스트 스트링을 사전 인덱스 스트링으로 변환  
7) X_train, y_train, X_test, y_test, word_to_index 리턴**  

In [3]:
# 학습데이터의 중복단어 갯수 확인
# .nunique()는 판다스에서 유니크한 값의 갯수를 알려줌
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [4]:
# 150000 => 146182로 중복 확인

# 테스트데이터의 중복단어 여부 확인
test_data['document'].nunique(), test_data['label'].nunique()

(49157, 2)

In [5]:
# 50000 => 49157로 중복 확인

# 데이터의 중복 제거 및 이상한 데이터 삭제
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data = train_data = train_data.dropna(how='any')
print('데이터 정리 한 학습용 리뷰 개수 :',len(train_data))

데이터 정리 한 학습용 리뷰 개수 : 146182


In [4]:
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    # [[YOUR CODE]]
    # 1-1) 학습데이터의 중복 제거
    train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
#     1-2) 학습데이터에서 NaN 결측치 제거
    train_data =train_data.dropna(how='any')
    
#     2-1) 테스트데이터의 중복 제거
    test_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
#     2-2) 테스트데이터의 NaN 결측치 제거
    test_data =train_data.dropna(how='any')
    
#     3-1) 학습데이터 한국어 토크나이저로 토큰화
    X_train = []
    for sentence in train_data['document']:
        # sentence에 있는 문장을 뜻을 가질수 있는 최소단위 '형태소'로 
        # 분리해서 토큰으로 가지고 있어줘
        temp_X = tokenizer.morphs(sentence) 
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어(stopwords) 제거
        # temp_X의 토큰을 X_train에 넣어줘.
        X_train.append(temp_X)

#     3-2) 테스트데이터 한국어 토크나이저로 토큰화
    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence)
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어(stopwords) 제거
        X_test.append(temp_X)

#     5) 사전word_to_index 구성
    # np.concatenate(배열) : 해당 배열을 오른쪽에 추가하기
    # .tolost() : 앞의 배열을 np.unit이 아닌 list로 바꿔줌
    # => words는 X_train의 배열을 가진 list가 됨.
    words = np.concatenate(X_train).tolist()
    # Counter(words)
    # words에서 단어들의 갯수를 세어 {단어1:갯수, 단어2:갯수, ...} 형태의
    # 딕셔너리를 형성함.
    # ex) words = [가, 나, 가, 다, 나, 가, 라] 일때
    # Counter(words)를 하면
    # 결과물로 ({가:3, 나:2, 다:1, 라:1})의 형태임.
    counter = Counter(words)
    # counter.most_common(10000-4)
    # conter에서 가장 많이 사용된 단어를 [(단어1, 횟수),(단어2, 횟수), ...]의 형태로
    # 정렬하여 가장 많이 나온 횟수부터 가장 적게나온 횟수 순으로 (10000-4)개를 정렬해준다.
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

#     6) 텍스트 스트링을 사전 인덱스 스트링으로 변환  
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    # map() 함수는 for문과 같은 반복문을 사용하지 않아도 
    # 지정한 함수로 인자를 여러번 전달해 그 결과를 list 형태로 
    # 뽑아 주는 유용한 함수이다.
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

#     7) X_train, y_train, X_test, y_test, word_to_index 리턴
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index


    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [7]:
# 데이터의 갯수 확인
print('학습용 리뷰 개수 :',len(train_data))
print('테스트용 리뷰 개수 :',len(test_data))

학습용 리뷰 개수 : 146183
테스트용 리뷰 개수 : 49158
